In [1]:
import numpy as np
import pandas as pd 
import random
import methods
import TimeSeries_Clustering
from tqdm import tqdm 
import matplotlib.pyplot as plt
import pandas as pd
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import sigma_gak, cdist_gak
from sklearn.metrics.cluster import adjusted_rand_score
import random
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
import xgboost
import gc
import Evrecsys
import sys
from itertools import combinations, groupby
from collections import Counter
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
import Apriori
import lightfm_form
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm import LightFM
import TimeSeries_Clustering
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
#Import Data
merchants = pd.read_csv('merchants.csv')
data = pd.read_csv('data_cards_0.csv',parse_dates=['purchase_date'])
test = pd.read_csv('test.csv', parse_dates=["first_active_month"])
train = pd.read_csv('train.csv', parse_dates=["first_active_month"])
card_list=data.card_id.unique()
test= test[test.card_id.isin(card_list)]
train= train[train.card_id.isin(card_list)]

In [3]:
train_df, val_df = train_test_split(train, test_size=0.3)

## Average Merchant

In [4]:
def avg_merch(train, val,test,data):
    train=train.merge(data, on='card_id', how='left')
    train_avgtarget = train.groupby(["merchant_id"])["target"].aggregate("mean").reset_index()
    train=train.merge(train_avgtarget, on='merchant_id', how='left')
    val=val.merge(data, on='card_id', how='left')
    val=val.merge(train_avgtarget, on='merchant_id', how='left').fillna(train_avgtarget.target.mean())
    pred=val.groupby(["card_id"])["target_y"].aggregate("mean").reset_index()
    pred=pred.merge(val[['card_id','target_x']], on='card_id',how='left').drop_duplicates()
    return np.sqrt(mean_squared_error(pred.target_y, pred.target_x))
    
    

In [5]:
avg_merch(train_df, val_df,test,data)

3.4259014375403916

## Last Merchant

In [6]:
def last_merch(train, val,test,data):
    train=train.merge(data, on='card_id', how='left')
    train_avgtarget = train.groupby(["merchant_id"])["target"].aggregate("mean").reset_index()
    val=val.merge(data, on='card_id', how='left')
    val=val.merge(train_avgtarget, on='merchant_id', how='left').fillna(train_avgtarget.target.mean())
    idx_new=val.groupby(['card_id'])['purchase_date'].transform(max) == val['purchase_date']
    pred=val[idx_new]
    return np.sqrt(mean_squared_error(pred.target_y, pred.target_x))
    
    

In [7]:
last_merch(train_df, val_df,test,data)

3.507330183890826

## Apriori

In [9]:
train_df=train_df.merge(data, on='card_id', how='left')
val_df=val_df.merge(data, on='card_id', how='left')
train_df = train_df.set_index('merchant_id')['card_id'].rename('item_id')
val_df = val_df.set_index('merchant_id')['card_id'].rename('item_id')

In [10]:
rules_i = Apriori.association_rules(train_df, 0.01)

Starting order_item:                  78797
Items with support >= 0.01:             856
Remaining order_item:                 78797
Remaining orders with 2+ items:        9171
Remaining order_item:                 66291
Item pairs:                             779
Item pairs with support >= 0.01:        779



/home/grecia/Dropbox/SKOLTECH/thesis/data_sets/ELO/Apriori.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  order_item = order_item.reset_index().as_matrix()


In [ ]:
train_pairs_gen_i = Apriori.get_item_pairs(train_orders_i)
train_pairs_i = Apriori.freq(train_pairs_gen_i).to_frame("freqAB")
train_pairs_i = train_pairs_i.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
train_pairs_i['pair'] = train_pairs_i.item_A.astype(str).str.cat(train_pairs_i.item_B.astype(str), sep='-')

In [ ]:
def apriori(train, test, return_pred, num_cluster):
    train_orders_i = train.set_index('merchant_id')['card_id'].rename('item_id')
    test_orders_i = test.set_index('merchant_id')['card_id'].rename('item_id')

    #item_name = train['product_id', 'product_name', 'aisle_id', 'department_id'].rename(columns={'product_id': 'item_id', 'product_name': 'item_name'})
    rules_i = Apriori.association_rules(train_orders_i, 0.01)
    #rules_final_i = Apriori.merge_item_name(rules_i, item_name).sort_values('lift', ascending=False)
    #display(rules_final_i)

    # Train set pairs
    train_pairs_gen_i = Apriori.get_item_pairs(train_orders_i)
    train_pairs_i = Apriori.freq(train_pairs_gen_i).to_frame("freqAB")
    train_pairs_i = train_pairs_i.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    train_pairs_i['pair'] = train_pairs_i.item_A.astype(str).str.cat(train_pairs_i.item_B.astype(str), sep='-')

    # Test set pairs
    test_pairs_gen_i = Apriori.get_item_pairs(test_orders_i)
    test_pairs_i = Apriori.freq(test_pairs_gen_i).to_frame("freqAB")
    test_pairs_i = test_pairs_i.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    test_pairs_i['pair'] = test_pairs_i.item_A.astype(str).str.cat(test_pairs_i.item_B.astype(str), sep='-')

    # Rules set pairs
    rules_i['pair'] = rules_i.item_A.astype(str).str.cat(rules_i.item_B.astype(str), sep='-')

    test_pair_set_i = set(np.unique(test_pairs_i.pair))
    train_pair_set_i = set(np.unique(train_pairs_i.pair))
    rules_pair_set_i = set(np.unique(rules_i.pair))

    # TP= Pairs that exist in a priori pred and test
    tp = len(list(test_pair_set_i & rules_pair_set_i))

    # TN= pairs that exists train set but not in test
    tn = len(list(test_pair_set_i - train_pair_set_i))

    # FN= Pairs that exists in test but not in a priori
    fn = len(list(rules_pair_set_i - test_pair_set_i))

    # FP= Pairs that exists in a priori but not in test
    fp = len(list(test_pair_set_i - rules_pair_set_i))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (recall * precision) / (recall + precision)
    print('APRIORI')
    return recall, precision, f1
